In [1]:
import os.path as osp
import torch
import numpy as np
from datasets import CardsDataset
from augmentations import SSDAugmentation, BaseTransform
from torchvision.models.detection import ssd300_vgg16 as ssd300
from mAP import intersection_over_union, mean_average_precision, f1_score
from IPython.display import clear_output

In [2]:
def eval(model, dataset, threshold=0.3):
    model.eval()
    list_pred_boxes = []
    list_truth_boxes = []
    mAP = 0
    ap = []
    
    for i in range(len(dataset)):
        print('Detect image {}/{} '.format(i+1, len(dataset)))
        
        img, label = dataset[i]
        prediction = model([img])
        
        truth_boxes = label['boxes']
        truth_labels = label['labels']
        pred_boxes = prediction[0]['boxes']
        pred_labels = prediction[0]['labels']
        scores = prediction[0]['scores']
        
        for j in range(len(truth_boxes)):
            list_truth_boxes.append([i, truth_labels[j], truth_boxes[j]])
        
        for j in range(len(pred_boxes)):
            list_pred_boxes.append([i, pred_labels[j], scores[j], pred_boxes[j]])
    
    threshs = np.arange(0.5, 1.0, 0.05)
    
    for thresh in threshs:
        ap.append(mean_average_precision(list_pred_boxes, list_truth_boxes, thresh))
    
    f1, rec, prec = f1_score(list_pred_boxes, list_truth_boxes, score_threshold=threshold)
    mAP = np.mean(ap)
    clear_output(wait=True)
    
    return mAP, ap, f1, rec, prec

In [3]:
home = osp.expanduser("~")

get_dir = '\project_SSD\data'
data_root = home+get_dir

NUM_CLASSES = 7
labels_names = ('backbone',
                'nine', 'ten', 'jack', 
                'queen', 'king', 'ace')
dataset = CardsDataset(root=data_root, mode='test', transform=BaseTransform())

weights = 'weights/SSD300Final.pth'
model = ssd300(num_classes=NUM_CLASSES, pretrained_backbone=True)
model.load_state_dict(torch.load(weights))

print()

D:\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [4]:
mAP, ap, f1, rec, prec = eval(model, dataset, 0.6)
print('mAP: {}; AP50: {}'.format(mAP, ap[0]))
print('Precision: {}; Recall: {}; F1: {}'.format(np.mean(prec), np.mean(rec), np.mean(f1)))

mAP: 0.18433085083961487; AP50: 0.35620370507240295
Precision: 0.39463947465022403; Recall: 0.3418028950691223; F1: 0.35937084188720364
